# sunpy (with astropy) tutorial

In [ ]:
import pathlib

import sunpy
sunpy.log.setLevel('ERROR')

DATA_DIR = pathlib.Path('./')
# In the unlikely event of network issues, uncomment this to use predownloaded data
# DATA_DIR = pathlib.Path('/home/jovyan/scratch_space/sunpy-tutorial/sunpy/')

In this notebook, we will learn about the basic parts of the `sunpy` package as well as how `sunpy` makes use of several core components of the `astropy` package. By the end of this tutorial you will know how to:

- Create and manipulate unitful quantities with `astropy.units`
- Create and do basic arithmetic with `astropy.time.Time` objects 
- Search for data from many different data providers with `sunpy`
- Load, manipulate, and visualize 2D images data with `sunpy.map.Map`
- Do basic coordinate tranformations with the `astropy.coordinates` framework, include solar coordinate frames
- Create and visualize time series data with `sunpy.timeseries.TimeSeries`

## Part 0 - `astropy.units`

Let us start with values and scientific units. Often when we reperesent unitful quantities in code, we assume a particular unit system, maybe providing context in a comment.

In [ ]:
speed = 100 # km/s
speed_kms = 100
speed_cms = speed_kms * 100

This doesn't scale and mistakes can be made. Google "Mars Climate Orbiter" for an example of when units go wrong.

`astropy` includes a powerful framework for units that allows users to attach units to scalars and arrays.
These quantities can be manipulated or combined while keeping track of the units.

For more information about the features presented below, please see the [astropy.units](http://docs.astropy.org/en/stable/units/index.html) documentation.

In [ ]:
import astropy.units as u

The primary goal of the `astropy.units` package is to be able to store, convert, display units in code.

In [ ]:
u.meter

In [ ]:
?u.m

In [ ]:
u.m.physical_type

Notice that meter also has a shorthand called "m".

You can create 1 unit, a range of units, and convert the unit.

In [ ]:
100 * u.meter, 

In [ ]:
[1, 2, 4, 8] * u.km

In [ ]:
(100 * u.meter).to(u.AA)

This system also ensures that arithmetic operations between quantities make sense.

In [ ]:
10 * u.meter + 100 * u.cm

In [ ]:
1 * u.meter + 1 * u.gram

In addition, there are fixed constants like G and c.

In [ ]:
from astropy.constants import G, c

<div class="alert alert-block alert-warning">
    <emph><u>EXERCISE:</u>
    <br>
    Calculate the Schwarzschild Black Hole Radius of the Sun
    </emph>
    $$R = \frac{{2GM}}{{c^2 }}$$
</div>

In [ ]:
# INSTRUCTOR BLOCK
R_sch = 2 * G * u.Msun / c**2
R_sch.to(u.earthRad)

In [ ]:
R_sch.to(u.km)

Quantities also support conversion to *equivalent* unit systems (but you have to tell it!).

In [ ]:
(500*u.nm).to(u.Hz)

For this to work, we need to tell `astropy.units` the specific assumption we want to make about how these units are related.

In [ ]:
(500*u.nm).to(u.Hz, u.spectral())

These unitful quantities can also be passed to `numpy` functions.

In [ ]:
import numpy as np

In [ ]:
np.sin(90 * u.degree)

In [ ]:
np.sin(30 * u.rad)

Lots of methods and functions in `sunpy` require values to have units attached to them.
The first case will be us downloading data.

## Part 1 - Data Search and Download with `Fido`

In [ ]:
from sunpy.net import Fido, attrs as a

### Overview of the `Fido` Unified Downloader

* Fido is sunpy's interface for searching and downloading solar physics data.
* It offers a unified interface for searching and fetching data irrespective of the underlying client or web service from where the data is obtained.
* Offers a way to search and accesses multiple instruments and all available data providers in a single query.
* It supplies a single, easy, consistent and *extendable* way to get most forms of solar physics data the community need 

Fido currently offers access to data available through:

 * **Virtual Solar Observatory (VSO)**
 * **Joint Science Operations Center (JSOC)**
 * **Individual data providers** from web accessible sources (http, ftp, etc)
 * **Heliophysics Events Knowledgebase (HEK)**
 * **Heliophysics Feature Catalogue (HELIO)**
 * Other sources via a plugin system
 
As described here Fido provides access to many sources of data through different `clients`, these clients can be defined inside sunpy or in other packages.

Lets print the current list of available clients within sunpy.

In [ ]:
Fido

### Searching for Data

sunpy uses specified *attributes* to search for data using Fido.
These search attributes can be combined together to construct data search queries, such as searching over a certain time period or for data from a specific instrument observing in a certain wavelength.

Different clients may have client-specific attributes, but the core attributes are:

* `a.Time`
* `a.Instrument`
* `a.Wavelength`

Let's use these different attributes to construct a query for our CME observation.

In [ ]:
cme_start = "2022-03-28T11:00"
cme_end = "2022-03-28T14:00"

In [ ]:
cme_time = a.Time(cme_start, cme_end)

What is this time object?

In [ ]:
cme_time

In [ ]:
cme_time.start

In [ ]:
 type(cme_time.start)

### Aside - `astropy.time.Time`

Python already has a built-in datetime package which handles standard dates, times, timezones and time deltas.

In [ ]:
import datetime

We can get the current datetime

In [ ]:
datetime.datetime.now(datetime.timezone.utc)

We can do some arithmetic with the datetime objects 

In [ ]:
datetime.datetime(2024,1,1) - datetime.timedelta(minutes = 1)

So why do we need `astropy.time`?

Two main areas:

 - Astronomical formats (e.g., Julian Date (JD), Modified JD (MJD))
 - Precise timing (e.g., a nanosecond over a Hubble Time)

Neither are supported by the standard datetime library.

So let us import the time objects from astropy

In [ ]:
from astropy.time import Time, TimeDelta

The MJD for a given date is the number of days to that date since Jan 1 4713 B.C. 00:00:00 (midnight).

In [ ]:
time = Time(58086.182, format='mjd')

In [ ]:
time

You can represent these `Time` objects in other formats.

In [ ]:
time.jd

In [ ]:
time.iso

In [ ]:
time.isot

We can also do arithmetic with `Time`

In [ ]:
time - TimeDelta(1, format='jd') # Defaults to days.

In [ ]:
time - 1*u.day

This applies to arrays as well.

In [ ]:
times = time + np.linspace(0, 1, 10) * u.day

In [ ]:
times

In [ ]:
times - times[0]

In [ ]:
(times - times[0]).to('day')

For nanoseconds, we have to adjust the precision

In [ ]:
time = Time('1999-01-01T00:00:00.123456789')
print(time)

In [ ]:
time.precision = 9
print(time)

In [ ]:
time.datetime

Notice that the nanoseconds are gone from the datetime version.

### Combining Search Attributes

We can inspect the instrument attribute to see what instruments are currently supported through sunpy. Here we can see the instrument name (i.e., the name to be passed to the `a.Instrument` attribute, the client from which the data is available to access, and the full name of the instrument.

In [ ]:
a.Instrument

We can combine our time and instrument attributes to search for AIA data within our selected time range using `Fido.search`

In [ ]:
Fido.search(cme_time & a.Instrument.aia)

We can further filter our results using the `Wavelength` search attribute.

In [ ]:
Fido.search(cme_time & a.Instrument.aia & a.Wavelength(304*u.angstrom))

In [ ]:
aia_query = cme_time & a.Wavelength(304*u.angstrom) & a.Instrument.aia & a.Sample(12*u.min)

In [ ]:
Fido.search(aia_query)

<div class="alert alert-block alert-warning">
    <emph><u>EXERCISE:</u>
    <br>
    We've written a query for the AIA data above. How would we write a query for EUVI data from STEREO-A for the same time range, cadence, and wavelength?
    </emph>
</div>

In [ ]:
# INSTRUCTOR BLOCK
stereo_query = cme_time & a.Wavelength(304*u.angstrom) & a.Instrument.secchi & a.Sample(12*u.min)

### Combining Queries

In addition to making queries for individual instruments, we can also logically combine queries for multiple instruments at once. For example, if we wanted to search for data from both AIA and SECCHI for the same time range and passband.

In [ ]:
Fido.search(cme_time, a.Instrument.aia | a.Instrument.secchi, a.Wavelength(304*u.angstrom), a.Sample(12*u.minute))

What if we also wanted to look for the GOES XRS data during this same interval?

GOES/XRS data does not have a "Wavelength" or "Sample" associated with it, but we can still combine the queries for all three of these instruments.

In [ ]:
aia_or_secchi = (a.Instrument.aia | a.Instrument.secchi) & a.Wavelength(304*u.angstrom) & a.Sample(12*u.minute)

In [ ]:
goes_query = a.Instrument.xrs & a.goes.SatelliteNumber(17) & a.Resolution('flx1s')

In [ ]:
combined_query = Fido.search(cme_time, aia_or_secchi | goes_query)

In [ ]:
combined_query

Note that we get a different table back for each of our combination of search parameters.

In [ ]:
len(combined_query)

In [ ]:
combined_query[0]

### Downloading Data

We can easily make a single download request from all of our different clients by passing in our combined query for AIA, EUVI and XRS.

In [ ]:
files = Fido.fetch(combined_query, path='data/{instrument}')

Let us see what we got!

In [ ]:
files

Now we have all of these files, what do we do with them?

## Part 2 - The `Map` Data Structure

Let us start with the 2D data files.
We will use glob to separate the files.

In [ ]:
aia_files = sorted((DATA_DIR / 'data/AIA/').glob('*.fits'))
stereo_files = sorted((DATA_DIR / 'data/SECCHI/').glob('*.fts'))

In [ ]:
aia_files

The most common data format used in Solar Physics for remote sensing instruments are FITS files, which consist of Header Data Unit (HDU) Pairs.
We can open these easily with astropy.

In [ ]:
from astropy.io import fits

hdulist = fits.open(aia_files[0])
hdulist

Here, it isn't important to worry about the first item in this list.
We want to access the second element and check the data and FITS header of the file.

In [ ]:
hdulist[1].data

We can visualize this data array just using `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot()
ax.imshow(hdulist[1].data, vmin=0, vmax=100)
# Quick note to remember that we do not need to use plt.show() to display the plot in a notebook.

The metadata for this file is all stored in the FITS header. A FITS header can be long and difficult to parse. For example, the names of the keys can only be 8 characters long and can often be quite cryptic.

In [ ]:
hdulist[1].header

While we can access the data and header directly, working like that can pose some challenges. For example, what pixel(s) corresponds to the region on the Sun I am interested in? What coordinate system is my image in? 

We need a way to tie the data and the metadata together to form one coherent unit.

### Creating a `Map`

We create a `sunpy.map.Map` object by passing in the FITS file for a single AIA and SECCHI (STEREO) observation.

In [ ]:
import sunpy.map

In [ ]:
m_aia = sunpy.map.Map(aia_files[6])

In [ ]:
m_stereo = sunpy.map.Map(stereo_files[6])

We can easily visualize a map after loading it using the quicklook functionality that is enabled by the notebook.

In [ ]:
m_stereo

Or using the `peek` method

In [ ]:
m_stereo.peek()

We will talk much more about visualizing maps later on.

### Attributes of a `Map`

`Map` provides a common interface to most 2D imaging solar datasets and provides several useful pieces of metadata.
`Map` is a container for holding your data and metadata (usually from the FITS header) together.

The `.meta` and `.data` attributes provide access to the metadata and underlying array of image data, respectively.

In [ ]:
m_aia.data

In [ ]:
m_aia.meta

These are very similar to the outputs we got earlier from parsing the FITS file directly with `astropy`.

This metadata can be terse, non-homogeneous, and sometimes difficult to parse.
`Map` provides several high-level attributes derived from the underlying raw metadata that expose a uniform interface to the metadata for each map.

In [ ]:
m_aia.wavelength

In [ ]:
m_stereo.instrument

Each `Map` object also holds the unit system that the image data is in, expressed in terms of an `astropy.unit.Unit` object.

In [ ]:
m_stereo.unit

### Aside - `astropy.coordinates`

#### Coordinate Information

Each `Map` exposes information about which coordinate system the image was taken in, including the location of the spacecraft that recorded that observation.
`sunpy` leverages and extends the powerful `astropy` coordinate framework with solar-specific coordinate frames. 
The `SkyCoord` object is the primary interface for working with coordinates.
`SkyCoord` provides a simple and flexible user interface for celestial coordinate representation, manipulation, and transformation between coordinate frames. 

In [ ]:
from astropy.coordinates import SkyCoord

In [ ]:
point = SkyCoord(0*u.deg, 0*u.deg, frame='icrs')
point

Here, the ICRS<a name="icrs-note"></a>[<sup>[1]</sup>](#icrs-note) is just one coordinate frame among many.
`sunpy` adds a number of solar coordinate frames which are automatically registered with `astropy` when you import `sunpy.coordinates`.

In [ ]:
point.galactic

This is a specific example but as this is provided without context, let us show how this works within `sunpy`.

<a name="icrs-note"></a>[<sup>[1]</sup>](#icrs-note) A resolution passed in 1997 established the International Celestial Reference System (ICRS), a high precision coordinate
system with its origin at the solar system barycenter and "a space fixed" (kinematically nonrotating) axes. 

#### A solar coordinate frame

An example of a solar-specific coordinate system is the Stonyhurst heliographic (HGS) coordinate system. 
The HGS system is defined with the following Cartesian axes:

* The origin is the center of the Sun
* The Z-axis (+90 degrees latitude) is aligned with the Sun’s north pole.
* The X-axis (0 degrees longitude and 0 degrees latitude) is perpendicular to the Z-axis such that the XZ-plane contains the Sun-Earth line.  That is, Earth is at 0 degrees longitude (but usually not at 0 degrees latitude).
* The Y-axis (+90 degrees longitude and 0 degrees latitude) is perpendicular to both the X-axis and the Z-axis in a right-handed fashion.

Then, the coordinate frame is the realization of this definition at a particular time, which defines the position/orientation of the Sun and the position of the Earth.

Let's create a frame for Stonyhurst heliographic coordinates using sunpy's [`HeliographicStonyhurst` class](https://docs.sunpy.org/en/stable/api/sunpy.coordinates.frames.HeliographicStonyhurst.html):

In [ ]:
from sunpy.coordinates import HeliographicStonyhurst

In [ ]:
time = '2022-03-28 11:00'
hgs_frame = HeliographicStonyhurst(obstime=time)
hgs_frame

#### Coordinates and different representations

A **coordinate** combines position data with a `SkyCoord`.

In [ ]:
# longitude, latitude, and distance from the origin
hgs_coord = SkyCoord(10*u.deg, 20*u.deg, 1*u.AU, frame=hgs_frame)
hgs_coord

The invidual components of our coordinate can be accessed as properties.

In [ ]:
hgs_coord.lon

In [ ]:
hgs_coord.lat

In [ ]:
hgs_coord.radius

This position data can have different **representations**, e.g., spherical components or Cartesian components.

In [ ]:
hgs_coord.cartesian

In [ ]:
hgs_coord.spherical

#### Observer-based frames

A number of coordinate frames are **observer-based**, which means that the coordinate frame itself is defined by the position of the observer.
For example, helioprojective Cartesian coordinates are aligned such that one axis is aligned with the Sun-observer line.

Let's use the above `HeliographicStonyhurst` coordinate as the observer for a [`Helioprojective` frame](https://docs.sunpy.org/en/stable/api/sunpy.coordinates.frames.Helioprojective.html), here for 2D helioprojective coordinates: $(\theta_x, \theta_y) = (123^{\prime\prime}, 456^{\prime\prime})$.

In [ ]:
from sunpy.coordinates import Helioprojective

In [ ]:
hpc_frame = Helioprojective(obstime=time, observer=hgs_coord)

In [ ]:
SkyCoord(123*u.arcsec, 456*u.arcsec, frame=hpc_frame)

Recall that all of the `sunpy.Map`s we created so far are defined in a Helioprojective frame.

### Using Coordinates with `Map`

For each `Map`, we can easily access what *coordinate frame* the observation corresponds to.

In [ ]:
m_aia.coordinate_frame

Similarly, we can look at the location of the observer (as defined by the position of the satellite at the time of the observation).

In [ ]:
m_aia.observer_coordinate

In [ ]:
m_stereo.observer_coordinate

We can plot these observer coordinates to show the relative position, in heliographic longitude, of each spacecraft, similar to the SolarMACH plot we showed in our previous notebook.

(**NOTE:** *It is not particularly important to understand the intricacies of the plotting code below. This is merely to show we can use the coordination information in each map to visualize the relative positions of the three spacecraft we are concerned with here.*)

In [ ]:
# Leave this code here
fig = plt.figure(figsize=(8, 8))
ax = plt.subplot(projection='polar')

# Plot the Sun
ax.plot(0, 0, marker='o', markersize=20, label='Sun', color='yellow')

# Plot the satellite locations
for m in [m_aia, m_stereo]:
    sat = m.observatory
    coord = m.observer_coordinate
    ax.plot(coord.lon.to('rad'), coord.radius.to(u.AU), 'o', label=sat)

ax.set_theta_zero_location("S")
ax.set_rlabel_position(90)
ax.set_rlim(0, 1.3)
ax.legend()

### The World Coordinate System

The World Coordinate System or WCS is a framework for transforming between pixel and world coordinates.

In [ ]:
m_aia.wcs

There is a lot of complexity in how the WCS is defined, but for our purposes here, it is just a mechanism for transforming between the *pixel* coordinate system of our image and the *world* coordinate system defined by the coordinate frame in which our image is defined.

We can use the associated `pixel_to_world` and `world_to_pixel` functions to transform between the world and pixel coordinates of our images.

In [ ]:
m_aia.wcs.pixel_to_world(0*u.pix, 0*u.pix)

In [ ]:
m_aia.bottom_left_coord

The `bottom_left_coord` is the *center* of the pixel in the bottom left corner of our image.

In [ ]:
m_aia.wcs.world_to_pixel(m_aia.bottom_left_coord)

Similarly, we can confirm that the `center` coordinate falls on the center of the map.

In [ ]:
m_aia.center

In [ ]:
m_aia.wcs.world_to_pixel(m_aia.center)

In [ ]:
m_aia.dimensions

Note that the center of our AIA image does not align with the center of the Sun!

In [ ]:
m_aia.wcs.world_to_pixel(SkyCoord(Tx=0*u.arcsec, Ty=0*u.arcsec, frame=m_aia.coordinate_frame))

<div class="alert alert-block alert-warning">
    <emph><u>EXERCISE:</u>
    <br>
    How would you find the position of the center of the STEREO map in the pixel coordinates of the AIA map?
    </emph>
</div>

In [ ]:
# INSTRUCTOR BLOCK
m_aia.wcs.world_to_pixel(m_stereo.center)

### Visualization

`Map` provides some additional "helpers" for plotting the associated image data with the correct projection based on the WCS.

At a minimum, this can be accomplished through the `.plot()` method.
It is important to note that this method supports many of the same arguments as `imshow` does.

In [ ]:
m_aia.plot()

This "automagically" creates a figure and an axis (with a projection based on the WCS of the map) and plots our map on that axis, with a colormap and normalization tailored for the specific map source.
All of this visualization is built on top of `matplotlib` and the `WCSAxes` capabilities provided by `astropy`.
However, as you can see, the resulting default scaling is not particularly useful.

Because all of this plotting capability is built on top of `matplotlib`, we can easily customize the various components of our plot.

In [ ]:
plt.figure(figsize=(8, 8))
m_aia.plot(vmin=0, vmax=500)
m_aia.draw_grid(lw=1, alpha=1)

While it is nice to have `matplotlib` create everything for us, it is easier to customize the plotting process if we create the figure and axis ourselves.
This can be important when we have multiple figures within a notebook.

So we will now  create a figure and axis and add the projection for the map.
We can also easily adjust the limits on our colorbar using the `clip_interval` key.

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=m_aia)
im = m_aia.plot(axes=ax, clip_interval=(5,99.9)*u.percent)
grid = m_aia.draw_grid(axes=ax, lw=1, alpha=1)
ax.set_title(r'A nicer AIA 304 $\mathrm{\AA}$ Plot')
ax.coords[0].set_axislabel('HPC Lon')
ax.coords[1].set_axislabel('HPC Lat')
# This is just an example, it isn't the most useful
grid['lon'].set_ticks([-30, -45, -60] * u.deg)
grid['lat'].set_ticks([-45, -60, -75] * u.deg)
fig.colorbar(im)

Or specify a new normalization altogether

In [ ]:
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization import LogStretch

In [ ]:
norm = ImageNormalize(vmin=0, vmax=50, stretch=LogStretch())

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=m_aia)
m_aia.plot(axes=ax,norm=norm)

<div class="alert alert-block alert-warning">
    <emph><u>EXERCISE:</u> <br>How would I change the colormap for the above plot?</emph>
</div>

In [ ]:
# INSTRUCTOR BLOCK
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=m_aia)
m_aia.plot(axes=ax,norm=norm, cmap='jet')

Using `matplotlib` combined with `WCSAxes`, we can build more complex, publication-quality visualizations.

(**NOTE:** It is not necessary to fully understand every intricacy of the plotting code below during the course of the tutorial. This is merely to show how `Map.plot` can be be used to make more complex plots.)

In [ ]:
# Leave this code here
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import numpy as np

fig = plt.figure(figsize=(20,10))

for i, m in enumerate([m_aia, m_stereo]):
    # Create the axis with the appropriate projection
    ax = fig.add_subplot(1,3,i+1,projection=m)

    # Add the plot to the axis
    im = m.plot(axes=ax, annotate=False, clip_interval=(1,99.9)*u.percent)

    # Make the HPC grid lines visible
    ax.coords.grid(alpha=1, ls='-')

    # Adjust the labels and ticks
    if i > 0:
        ax.coords[1].set_auto_axislabel(False)
    else:
        ax.coords[1].set_axislabel('Solar-Y')
    ax.coords[0].set_axislabel('Solar-X')
    ax.coords[1].set_ticklabel(rotation=90,)

    # Put a label on each plot
    ax.text(m.data.shape[1]//2, m.data.shape[0]*.97, m.observatory,
            color='w',
            horizontalalignment='center',
            verticalalignment='top',
            fontsize=14)

    # Add a colorbar to the top of each plot
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('top', size='4%', pad=0.2, axes_class=plt.Axes)
    fig.colorbar(im, cax=cax, orientation='horizontal')

    cax.xaxis.set_ticks_position("top")
    cax.xaxis.set_tick_params(direction='in')

plt.subplots_adjust(wspace=0.1)

#### Overplotting Coordinates on a `Map`

Let's use our newfound knowledge of coordinates to plot the positions of the detected flares around the time we know that the CME initiated.
To find the metadata for any flares that were detected at this time, we'll again use `Fido` to query the Heliophysics Event Knowledgebase (HEK).

> The Heliophysics Events Knowledgebase (HEK) system is being developed to help solar and heliospheric researchers locate features and events of interest to their science topics.

This can be done by using the HEK client specific attributes `a.hek.attrs`.
We'll choose only flare events who have a GOES class above C2.5

In [ ]:
hek_result = Fido.search(
    a.Time(cme_start, cme_end),
    a.hek.EventType('FL'),
    a.hek.FL.GOESCls > 'C2.5'
)

The HEK keeps track of a lot of information for each event in the database.

In [ ]:
hek_result['hek']

Let's filter this down to the start, end, and peak times of the flare, the GOES classification, and the coordinates of the flare on the disk.

In [ ]:
flare_table = hek_result['hek'][
    'event_starttime',
    'event_peaktime',
    'event_endtime',
    'fl_goescls',
    'hpc_x',
    'hpc_y',
]

In [ ]:
flare_table

To further understand whether these flares in the HEK database correspond to the observed CME, we can plot the positions of the flares that occurred near the start of our observing interval on our AIA image.
We'll do this by first constructing a coordinate for each flare using the HPC positions returned by the HEK.
Though not explicitly stated here, these coordinates are computed assuming an Earth-based observer at the start time of the event.
We can get a `SkyCoord` denoting the position of Earth using the `sunpy.coordinates.get_earth` function.

In [ ]:
from sunpy.coordinates import get_earth

In [ ]:
# Leave this code here
flare_coords = []
for fl in flare_table:
    earth = get_earth(fl['event_starttime'])
    hpc_frame = Helioprojective(obstime=fl['event_starttime'], observer=earth)
    hpc_coord = SkyCoord(Tx=fl['hpc_x']*u.arcsec, Ty=fl['hpc_y']*u.arcsec, frame=hpc_frame)
    flare_coords.append(hpc_coord)

Finally, let's overplot these coordinates on our AIA and EUVI images.
The `plot_coord` command takes in a `SkyCoord` object and automatically transforms the coordinate to the coordinate system defined by the WCS on that axis.

In [ ]:
# Leave this code here
fig = plt.figure(figsize=(15,7.5))
for i,m in enumerate([m_aia, m_stereo]):
    ax = fig.add_subplot(1,2,i+1, projection=m)
    m.plot(axes=ax,clip_interval=(25,99.5)*u.percent)
    for fl in flare_coords:
        ax.plot_coord(fl, marker='X', color='C0', markersize=15)

We find that one of the flares queried from the HEK coincides with our observed CME while the other is offset to the south.

#### Animations with `MapSequence`

In addition, the `MapSequence` container provides a data container for holding multiple maps, e.g. when you have a sequence of maps taken at successive times.
We can create `MapSequence` objects by passing in our list of files and the `sequence=True` keyword argument.

In [ ]:
stereo_seq = sunpy.map.Map(stereo_files, sequence=True)

In [ ]:
stereo_seq

The `MapSequence` can be indexed to return the individual `Map` objects at each time step.

One of the most useful features of a `MapSequence` is the ability to create coordinate-aware visualizations of our stack of `Map` objects.
To do this, we'll first create a a colormap normalization appropriate to the range of the data for every map in our stack.

In [ ]:
from astropy.visualization import AsymmetricPercentileInterval

vmin, vmax = AsymmetricPercentileInterval(1, 99.5).get_limits(stereo_seq.as_array())
norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=LogStretch())

The `plot` method on our `MapSequence` object now returns an animation rather than a simple static plot.

In [ ]:
plt.figure(figsize=(10,10))
stereo_ani = stereo_seq.plot(norm=norm)

In [ ]:
from IPython.display import HTML

HTML(stereo_ani.to_jshtml())

### Basic Image Manipulation

There are several methods on the `Map` object that provide capabilities for doing basic image manipulation in combination with the coordinate information attached to each `Map`.

#### Rotate

The `.rotate` method applies a rotation in the image plane, i.e. about an axis out of the page. 
In the case where we do not specify an angle (or rotation matrix), the image will be rotated such that the world and pixel axes are aligned.
In the case of an image in helioprojective coordinate system, this means that solar north will be aligned with the y-like pixel axis of the image.

In [ ]:
m_stereo_rot = m_stereo.rotate(missing=m_stereo.min())

By default, any missing values will be filled with "NaN". Here, we specify `missing` as the minimum intensity value of the map.

In [ ]:
fig = plt.figure(figsize=(11,5), layout='constrained')

ax = fig.add_subplot(121,projection=m_stereo)
m_stereo.plot(axes=ax, vmin=800, vmax=5000)
ax.coords.grid(alpha=1, ls='-')

ax = fig.add_subplot(122,projection=m_stereo_rot)
m_stereo_rot.plot(axes=ax, vmin=800, vmax=5000)
ax.coords.grid(alpha=1, ls='-')

This rotation is also reflected in the updated metadata of the rotated image.

In [ ]:
m_stereo.rotation_matrix

In [ ]:
m_stereo_rot.rotation_matrix

Additionally, one can also specify some arbitrary angle to rotate the image by.
Note that this angle is relative to the current orientation of the image.

<div class="alert alert-block alert-warning">
    <emph><u>EXERCISE:</u>
    <br>
    How would you rotate the image such that there is exactly a 45 degree orientation between the world and pixel axes?
    </emph>
</div>

In [ ]:
# INSTRUCTOR BLOCK
m_stereo_45 = m_stereo.rotate(missing=m_stereo.min()).rotate(angle=45*u.degree, missing=m_stereo.min())

fig = plt.figure(figsize=(5,5))

ax = fig.add_subplot(projection=m_stereo_45)
m_stereo_45.plot(axes=ax, vmin=800, vmax=5000)
ax.coords.grid(alpha=1, ls='-')

#### Cropping Images

We commonly want to pare down our full field-of-view to a particular region of interest.
With a map, we can do this using the `submap` method.
We can specify the region of our submap using world coordinates as specified by a `SkyCoord`.
We will specify these coordinates in Heliographic Stonyhurst (HGS) coordinates.
From the animation of the STEREO data above, we can identify approximately where the CME was launched from and crop our image around that region.

In [ ]:
bottom_left = SkyCoord(lon=-20*u.deg, lat=-5*u.deg, radius=1*sunpy.sun.constants.radius,
                       frame='heliographic_stonyhurst', obstime=m_aia.date)
top_right = SkyCoord(lon=30*u.deg, lat=35*u.deg, radius=1*sunpy.sun.constants.radius,
                     frame='heliographic_stonyhurst', obstime=m_aia.date)

In [ ]:
m_stereo_cropped = m_stereo.submap(bottom_left, top_right=top_right)

In [ ]:
m_stereo_cropped

The coordinates for our cutout can also be specified in pixel coordinates.

<div class="alert alert-block alert-warning">
    <emph><u>EXERCISE:</u> <br>The coordinates for our cutout can also be specified in pixel coordinates. Find the corners of our cutout in pixel coordinates and then create the same submap using those pixel coordinates.
</emph>
</div>

In [ ]:
# INSTRUCTOR BLOCK
bl_pix = m_stereo.wcs.world_to_pixel(m_stereo_cropped.bottom_left_coord)
tr_pix = m_stereo.wcs.world_to_pixel(m_stereo_cropped.top_right_coord)
m_stereo.submap(bl_pix*u.pixel, top_right=tr_pix*u.pixel)

## Part 3 - The `Timeseries` Data Structure

In addition to `Map` for 2D image data, `sunpy` also provides a container for tabular time series data through the `TimeSeries` class.
We can create a `TimeSeries` object in a very similar manner to how we create a `Map` object.

Let's look at the corresponding GOES XRS data that we downloaded above.

In [ ]:
goes_files = sorted((DATA_DIR / 'data/XRS/').glob('*.nc'))

In [ ]:
import sunpy.timeseries

In [ ]:
ts = sunpy.timeseries.TimeSeries(goes_files)

In [ ]:
ts

As with `Map`, `TimeSeries` acts as a container for the data + metadata. We can access each component individually.

In [ ]:
ts.meta

The `TimeSeries` object can also be converted to other formats like an `astropy` `Table` object

In [ ]:
ts.to_table()

or a `pandas` `DataFrame`

In [ ]:
ts.to_dataframe()

There are also a number of attributes on each `TimeSeries` derived from the data/metadata.

In [ ]:
ts.columns

In [ ]:
ts.observatory

In [ ]:
ts.units

### Slicing and Visualizing `TimeSeries`

Note that this intensity `TimeSeries` spans 24 h of observation time and recall that we are only interested in the ~3 h interval in which the CME is visible in the 304 channel.

We can truncate our timeseries around the times of interest.
To do this, we can actually use the `date` property on our first and last EUI map from our sequence.

In [ ]:
from sunpy.time import TimeRange

In [ ]:
ts_cme = ts.truncate(TimeRange(stereo_seq[0].date, b=stereo_seq[-1].date))

And then do a quicklook on our lightcurve.

In [ ]:
ts_cme

As expected, we find that there is a flare occurring right around the time the CME occurs. This should not be surprising as we saw from the AIA data that the CME was Earth-directed such that GOES was well-position to observed the flare.

We can also zoom in a bit on the beginning of the flare.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ts_cme.plot(axes=ax)
ax.set_xlim('2022-03-28 11:00', '2022-03-28 11:30')

As expected, we find that the flare, as detected by GOES, begins just before the eruption is seen by STEREO at 11:20.

### Combining GOES and HEK 

Additionally, let's load back in our GOES XRS timeseries over the entire time interval of interest.
Let's plot the times of the flares on top of our untruncated timeseries.
We'll indicate each flare with a shaded blue region and our original time interval of interest that we used to query our imaging observations with orange.

In [ ]:
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)
ts.plot(axes=ax)
ax.axvspan(cme_start, cme_end, alpha=0.25, color='C1')
for fl in flare_table:
    ax.axvspan(fl['event_starttime'].iso, fl['event_endtime'].iso,
               color='C0', alpha=0.25)

## Conclusion

In this notebook, we've learned how to use `sunpy`, including functionality from `astropy`, to search for, download, load and manipulate both 2D image as well as time series data. In particular, we learned how to:

- Create and manipulate unitful quantities with `astropy.units`
- Create and do basic arithmetic with `astropy.time.Time` objects 
- Search for data from many different data providers with `sunpy`
- Load, manipulate, and visualize 2D images data with `sunpy.map.Map`
- Do basic coordinate tranformations with the `astropy.coordinates` framework, include solar coordinate frames
- Create and visualize time series data with `sunpy.timeseries.TimeSeries`

Many more examples of how to use `sunpy` to accomplish these and other similar tasks can be found in the [`sunpy` example gallery](https://docs.sunpy.org/en/stable/generated/gallery/).